In [3]:
from termcolor import cprint

def print_board(board, bolded=None):
    """A debugging function to print you board in a pretty way"""
    n = len(board)
    # For every row but the last
    for row_idx, row in enumerate(board[:-1]):
        # Print the row as a string with a line below
        if bolded and row_idx==bolded[0]:
            cprint("|".join(row[:bolded[1]]), None, attrs=["underline"], end='|' if bolded[1] != 0 else '')
            cprint(row[bolded[1]], None, attrs=["underline", "bold"], end='')
            if bolded[1] != len(row) - 1:
                cprint("|" + "|".join(row[bolded[1]+1:]), None, attrs=["underline"], end='')
            print()
        else:
            cprint("|".join(row), None, attrs=["underline"])
    row = board[-1]
    if bolded and bolded[0] == len(board) - 1:
        print("|".join(row[:bolded[1]]), end='|' if bolded[1] != 0 else '')
        cprint(row[bolded[1]], None, attrs=["bold"], end='')
        if bolded[1] != len(row) - 1:
            cprint("|" + "|".join(row[bolded[1]+1:]), None, end='')
        print()
    else:
        print("|".join(row))


def other_stone(stone):
    return "X" if stone == "O" else "O"

def consecutive_k(row, k, stone):
    desired_row = [stone] * k
    return sum(desired_row == row[i:i+k] for i in range(len(row) - k + 1))

def get_downdiag(board, row_idx, col_idx):
    return [board[row_idx + i][col_idx + i] for i in range(min(len(board) - row_idx, len(board) - col_idx))]

def get_updiag(board, row_idx, col_idx):
    return [board[row_idx - i][col_idx + i] for i in range(min(row_idx + 1, len(board) - col_idx))]

def winner_stone(board, stone):
    k_count = 0
    k = 5
    for row in board:
        row = list(row)
        k_count += consecutive_k(row, k, stone)

    for col_idx in range(len(board)):
        bl = [row[col_idx] for row in board]
        k_count += consecutive_k(bl, k, stone)

    for row_idx in range(len(board)):
        bl = get_updiag(board, row_idx, 0)
        k_count += consecutive_k(bl, k, stone)
        bl = get_downdiag(board, row_idx, 0)
        k_count += consecutive_k(bl, k, stone)

    for col_idx in range(len(board[0])):
        bl = get_updiag(board, len(board) - 1, col_idx)
        k_count += consecutive_k(bl, k, stone)
        bl = get_downdiag(board, 0, col_idx)
        k_count += consecutive_k(bl, k, stone)
    
    return k_count > 0

def complete_board(board):
    return winner_stone(board, "X") or winner_stone(board, "O") or not any('-' in row for row in board)


In [4]:
import random
import heapq
import numpy as np
from tqdm.notebook import tqdm
import copy

class Strategy:
    def __init__(self, stone, eval_function, max_depth=1):
        self.stone = stone
        self.opponent_stone = other_stone(stone)
        self.max_depth = max_depth
        self.nodes = 0
        self.pruned = 0
        self.eval_function=eval_function 
    
    def alphabeta_search(self, board, depth, stone, alpha, beta):
        if depth == 0 or complete_board(board):
            score = self.eval_function(board, stone)
            score = -score if stone == self.opponent_stone else score
            self.nodes += 1
            return score, (None, None)
        
        row_arr, col_arr = np.where(board == '-')
        open_spaces = list(zip(row_arr, col_arr))
        if stone == self.stone:
            best_score = -np.infty
            best_move = (None, None)
            lop = len(open_spaces)
            c = 0
            for row, col in open_spaces:
                new_board = copy.deepcopy(board)
                new_board[row][col] = stone
                score, move = self.alphabeta_search(new_board, depth-1, other_stone(stone), alpha, beta)
                best_score = max(score, best_score)
                if best_score >= beta:
                    self.pruned += (lop - c)**depth
                    break
                elif best_score > alpha:
                    best_move = (row, col)
                    alpha = best_score
        else:
            best_score = np.infty
            best_move = (None, None)
            lop = len(open_spaces)
            c = 0
            for row, col in open_spaces:
                c += 1
                new_board = copy.deepcopy(board)
                new_board[row][col] = stone
                score, move = self.alphabeta_search(new_board, depth-1, other_stone(stone), alpha, beta)
                best_score = min(score, best_score)
                if best_score <= alpha:
                    self.pruned += (lop - c)**depth
                    break
                elif best_score < beta:
                    best_move = (row, col)
                    beta = best_score
                
        self.nodes += 1
        return best_score, best_move
        
                    
    def get_move(self, board, max_nodes = 100):
        score, move = self.alphabeta_search(board, self.max_depth, self.stone, -np.inf, np.inf)
        return move
            


In [6]:
def random_eval(_1, _2):
    return random.random()

def check_col(board, char):
  char_won = False
  length = len(board)
  for i in range(length):
    char_count = 0
    for j in range(length):
      if board[j][i] == char:
        char_count = char_count + 1
      else:
        char_count = 0
    if char_count == length:
      char_won = True
  return char_won

def check_row(board, char):
  char_won = False
  length = len(board)
  for i in range(length):
    char_count = 0
    for j in range(length):
      if board[i][j] == char:
        char_count = char_count + 1
      else:
        char_count = 0

    if char_count == length:
      char_won = True
  return char_won


def check_diagonal_up_down(board, char):
  char_won = False
  length = len(board)
  row = 0
  col = 0
  char_count = 0
  # if board[row][col] == char:
  #   char_count = char_count + 1
  while row < length and col < length:
    if board[row][col] == char:
      #print("row", row, "col", col)
      char_count = char_count + 1
    row = row + 1
    col = col + 1
    #print("char_count", char_count)
    if char_count == length:
      char_won = True
  return char_won

def check_diagonal_down_up(board, char):
  char_won = False
  length = len(board)
  row = length - 1
  col = 0
  char_count = 0
  #char_count = char_count + 1
  while row >= 0 and col < length:
    if board[row][col] == char:
      #print("row", row, "col", col)
      char_count = char_count + 1
    row = row - 1
    col = col + 1
    if char_count == length:
      char_won = True
    #print("char_count", char_count)
  return char_won

def game_state(board):
  x_won = False
  o_won = False
  x_won_row = check_row(board, "X")
  x_won_col = check_col(board, "X")
  x_won_updown_diag = check_diagonal_up_down(board, "X")
  x_won_downup_diag = check_diagonal_down_up(board, "X")
  o_won_row = check_row(board, "O")
  o_won_col = check_col(board, "O")
  o_won_updown_diag = check_diagonal_up_down(board, "O")
  o_won_downup_diag = check_diagonal_down_up(board, "O")

  if x_won_row or x_won_col or  x_won_updown_diag or x_won_downup_diag:
    x_won = True
  if o_won_row or o_won_col or o_won_updown_diag or o_won_downup_diag:
    o_won = True

  if x_won == False and o_won == False:
    return None
  if x_won == True and o_won == True:
    return "ERROR"
  if x_won == True:
    return "X"
  if o_won == True:
    return "O"


def student_eval(board, stone):
  if stone == "X":
    if game_state(board) == stone:
      return 999
    if game_state(board) == "O":
      return -999
  if stone == "O":
    if game_state(board) == stone:
      return 999
    if game_state(board) == "X":
      return -999

  length = len(board)
  is_empty = True
  for i in range(length):
    for j in range(length):
      if board[i][j] != "" or board[i][j] != "-":
        is_empty = False
  if is_empty == True:
    return 0

  
    # Your code goes here!
    #return random.random() # Replace this!

In [5]:
def play_game(board, strategy_1, strategy_2):
    player1_turn = True
    turn_count = 0
    n = len(board)
    m = len(board[0])
    while not complete_board(board) and turn_count < m*n:
        if player1_turn:
            row, col = strategy_1.get_move(board)
            board[row][col] = "X"
            if winner_stone(board, "X"):
                print("Student won!")
        else:
            row, col = strategy_2.get_move(board)
            board[row][col] = "O"
            if winner_stone(board, "O"):
                print("Computer won!")
        turn_count += 1
        print(turn_count, "X" if player1_turn else "O", f"{row},{col}")
        print_board(board, (row, col))
        board[row][col] = "X" if player1_turn else "O"
        player1_turn = not player1_turn
    if not winner_stone(board, "O") and  not winner_stone(board, "X"):
        print("Tie!")
    return complete_board(board)



big_board = np.array([list(x) for x in """
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-""".replace("|", "").split('\n')[1:]])

small_board = np.array([list(x) for x in """
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-""".replace("|", "").split('\n')[1:]])

p1 = Strategy("X", student_eval, max_depth=1) # Do not change the search depths!
p2 = Strategy("O", random_eval, max_depth=1)
play_game(small_board, p1, p2) # Start with the small board, it is better for debugging.

TypeError: ignored